## <font color='darkblue'>Preface</font>
([article source](https://blog.daftcode.pl/covariance-contravariance-and-invariance-the-ultimate-python-guide-8fabc0c24278)) <b><font size='3ptx'>This blog post is about covariance, contravariance, and invariance of Python types. I define these concepts and explain them in detail. Every step is accompanied by a fairly straightforward code snippet. I aim to show that the knowledge of these concepts helps to write more reliable code and it is beneficial to all Python programmers.</font></b>

This is the third part of a blog post series about contravariance and issues related to it. This part is self-contained, though, and can be read separately. The first two parts are devoted to understanding ([s02e01](https://blog.daftcode.pl/csi-python-type-system-episode-1-1c2ee1f8047c)) and fixing ([s02e02](https://blog.daftcode.pl/csi-python-type-system-episode-2-baf5168038c0)) a specific contravariance-related bug.

There are some formalisms in this post, yet they are quite straightforward. <b>I use `<:` symbol, which reads “is a subtype of”</b> (i.e. subtype is always at the pointy end 🗡 😉). If you are not sure what subtyping is or how it differs from subclassing, just read [this](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af#3df5) and [this](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af#70a9) part of s01e01 blog post.

In [13]:
from typing import Callable, List, Tuple

class Animal: ...
  
class Dog(Animal): ...

## <font color='darkblue'>Covariance</font>

### <font color='darkgreen'>Tuple</font>
Let’s say we have a tuple of <b><font color='blue'>Dog</font></b>s and a tuple of <b><font color='blue'>Animal</font></b>s. Their types are `Tuple[Dog, ...]` and `Tuple[Animal, ...]`, respectively (<font color='brown'>for a reminder about `Tuple` type, see [here](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af#0c1d)</font>). Every object with the type of `Tuple[Dog]` can be safely assigned to a variable of the type `Tuple[Animal]`, but not the other way around:

In [3]:
an_animal: Animal = Animal()
lassie: Dog = Dog()
snoopy: Dog = Dog()

animals: Tuple[Animal, ...] = (an_animal, lassie)
dogs: Tuple[Dog, ...] = (lassie, snoopy)

dogs = animals  # mypy error:
# Incompatible types in assignment (expression has type 
#   "Tuple[Animal, ...]", variable has type "Tuple[Dog, ...]")

Why the error is reported? It's because <b>animals tuple could contain other animals than dogs</b>. (<font color='brown'>And in our code in fact it does: `an_animal`.</font>) Therefore, the type of dogs, which is `Tuple[Dog, ...]`, could be compromised if we assigned an object of `Tuple[Animal, ...]` type to it. (<font color='brown'>And in our code in fact it is assigned.</font>) 
<b><font color='orange'>其實這道理就跟 `下雨 -> 地濕` , 但是反過來不對: `地濕 -> 下雨`</font></b>

On the other hand, [**mypy**](https://mypy.readthedocs.io/en/stable/) is happy with `animals = dogs` assignment. It’s because every element of `dogs` is a <font color='blue'><b>Dog</b></font> and, due to tuple’s immutability, cannot be anything else. Therefore, it can be used where an object of the type <b><font color='blue'>Animal</font></b> is expected, as <font color='blue'><b>Dog</b></font> is a subtype of <b><font color='blue'>Animal</font></b>. Thus `Tuple[Dog, ...] <: Tuple[Animal, ...]`.

More generally:
> `Tuple[SubType, ...] <: Tuple[SuperType, ...]`, <br/>
> for `SubType <: SuperType`.

Similarly, in the case of multiple-type tuples:
> `Tuple[SubType1, SubType2, etc.] <: Tuple[SuperType1, SuperType2, etc.]`,<br/>
> for `SubType1 <: SuperType1`, `SubType2 <: SuperType2` (etc.).

### <font color='darkgreen'>Definition of Covariance</font>
<b>The property of `Tuple` type we have just discovered is called a <font color='darkblue'>covariance</font></b>. Or, more precisely: <b>`Tuple` is covariant in all its arguments</b>. The formal definition of the <font color='darkblue'><b>covariance</b></font> is as follows (<font color='brown'>I slightly modified the definition from [mypy docs](https://mypy.readthedocs.io/en/latest/generics.html#variance-of-generic-types)</font>):
> Generic type `GenType[T, ...]` is covariant in type variable `T` <br/>
> if `GenType[SubType, ...] <: GenType[SuperType, ...]`, <br/>
> for `SubType <: SuperType`.

A generic type is, basically, a type that takes other types as its parameters, in square brackets, like `Tuple`, `List`, `Union` (<font color='brown'>see [mypy docs](https://mypy.readthedocs.io/en/latest/generics.html#generics)</font>). In Python, most immutable containers are covariant. `Tuple`s and [`frozenset`s](https://docs.python.org/3/library/stdtypes.html#frozenset) (<font color='brown'>their type is [FrozenSet](https://docs.python.org/3/library/typing.html#typing.FrozenSet)</font>) are the most significant ones. But there are other covariant types as well.

### <font color='darkgreen'>Union</font>
[**Union**](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af#9fbc) type is covariant in all its arguments:
> `Union[SubType1, SubType2, etc.] <: Union[SuperType1, SuperType2, etc.]`, <br/>
> for `SubType1 <: SuperType1`, `SubType2 <: SuperType2` (etc.).

The covariance of [**Union**](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af#9fbc) basically works the same way as the covariance of `Tuple`. Let’s say we have two pairs of types: `Dog <: Animal` and `Meat <: Food`. It’s safe to use an object of the type `Union[Dog, Meat]` where an object of the type `Union[Animal, Food]` is expected, but not the other way around. So, it’s okay to assign an object of the type `Union[Dog, Meat]` to a variable of the type `Union[Animal, Food]`, but not vice versa.

### <font color='darkgreen'>Return Type of Callable</font>
The case of [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) is a little bit more interesting. We can look at [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) type from two points of view: types of its arguments and the type of its return value.

As for the type of return value, let’s look at the following functions. Both are getting dogs and animals from, say, external services.

In [5]:
def get_dog() -> Dog:  # the type is: Callable[[], Dog]
    dog = Dog()
    return dog

  
def get_animal() -> Animal:  # the type is: Callable[[], Animal]
    animal = Animal()
    return animal

Now, it’s safe to use `get_dog` (<font color='brown'>returning a</font> <font color='blue'><b>Dog</b></font>) wherever `get_animal` (<font color='brown'>returning an</font> <font color='blue'><b>Animal</b></font>) is expected:
```python
some_animal: Animal = get_animal()
```

Type of `some_animal` is <font color='blue'><b>Animal</b></font>. `get_animal` can safely be switched with `get_dog`, then. It's because it will return a <font color='blue'><b>Dog</b></font> which can be safely used wherever <font color='blue'><b>Animal</b></font> is used because `Dog <: Animal`. So `Callable[[], Dog] <: Callable[[], Animal]`, or generally:
> `Callable[[], SubType] <: Callable[[], SuperType]`, <br/>
> for `SubType <: SuperType`.

So [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) is covariant in the return type.

<b>In more general terms, anything that provides something — producer, factory, constructor, writer, etc. — is covariant in the type of the objects it provides. It needs to be read-only with regard to the provided objects, though</b>.

## <font color='darkblue'>Contravariance</font>

### <font color='darkgreen'>Definition of Contravariance</font>
Contrary term to “covariance” is “contravariance”. It’s defined as follows:
> Generic type `GenType[T, ...]` is contravariant in type variable `T` <br/>
> if `GenType[SuperType, ...] <: GenType[SubType, ...]`, <br/>
> for `SubType <: SuperType`.

The definition is almost the same as of “covariance”, but with <: relation switched. Let’s get back to [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc).
> `Callable[[SuperType], None] <: Callable[[SubType], None]`, <br/>
> for `SubType <: SuperType`.

<br/>

<b><font color='orange'>My understanding: </font></b>
> 你的貓生病了, 本來想找 專門看貓的醫院 (`SubType`), 結果 只有 一般動物醫院 (`SuperType`) - Ok <br/>
> 你個狗生病了, 來來想找 一般動物醫院 (`SuperType`), 結果 只有 專門看鳥的醫院 (`SubType`) - Not Ok

### <font color='darkgreen'>Arguments of Callable</font>
The other way of looking at [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) is from the types of its arguments point of view. It turns out — and we will see why in a second — that in the case of function’s arguments, subtyping works the other way around compared to plain objects:
> `Callable[[SuperType], None] <: Callable[[SubType], None]`, <br/>
> for `SubType <: SuperType`.

**So,** [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) **is contravariant in the argument types**. Take note that for [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) contravariance works for an arbitrary number of arguments, as long as they are in subtyping relationships in a pairwise fashion:
> `Callable[[SuperType1, SuperType2, etc.], None] <: Callable[[SubType1, SubType2, etc.], None]`, <br/>
> for `SubType1 <: SuperType1`, `SubType2 <: SuperType2` (etc.).

Let’s find out why [**Callable**](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9#e1dc) is not covariant but contravariant in the argument types, then. (<font color='brown'>The following part is the essence of [s02e01](https://medium.com/@pawel.swiecki/csi-python-type-system-1c2ee1f8047c) post. Look there for more examples and explanations.</font>)

Take a look at the following code:

In [6]:
class Animal: ...
class Dog(Animal): ...
class Kangaroo(Animal): ...

  
def animal_run(animal: Animal) -> None:
    # its type is `Callable[[Animal], None]`
    print('An animal is running!')

    
def dog_run(dog: Dog) -> None:
    # its type is `Callable[[Dog], None]`
    print('A dog is running!')

Can we assign `animal_run = dog_run`? That is: can we safely use `dog_run` wherever `animal_run` is expected? If we could, it’d mean that `Callable[[Dog], None]` is a subtype of `Callable[[Animal], None]`. It turns out we cannot safely make the assignment. If we could, it would be possible to use `dog_run` on an incompatible object, say, a `Kangaroo`.

In [9]:
def make_animal_run(
        an_animal: Animal,
        run_function: Callable[[Animal], None],
) -> None:
    run_function(an_animal)

bob_a_kangaroo: Kangaroo = Kangaroo()

make_animal_run(bob_a_kangaroo, dog_run)  # mypy error: 
# Argument 2 to "make_animal_run" has incompatible type 
#   "Callable[[Dog], None]"; expected "Callable[[Animal], None]"

A dog is running!


Why [**mypy**](https://mypy.readthedocs.io/en/stable/) reports this error? So, `make_animal_run` gets two arguments: Bob, a Kangaroo (<font color='brown'>and it indeed is type-safe to pass him here, since `Kangaroo <: Animal`</font>) and a `dog_run` — a `Callable[[Animal], None]`. Now, inside `make_animal_run` function `dog_run` function is to be called on kangaroo Bob. It’s incorrect since `dog_run` can only be called on a <b><font color='blue'>Dog</font></b> and its subtypes. Therefore, we cannot safely pass `dog_run` to `make_animal_run`. For this reason, `Callable[[Dog], None]` is not a subtype of `Callable[[Animal], None]`. And this, in turn, leads to the conclusion: <b>Callable is not covariant in the argument types</b>.
    
What about the contravariance? Can we assign `dog_run = animal_run`? That is: can we safely use `animal_run` wherever `dog_run` is expected?

In [10]:
def make_dog_run(
        a_dog: Dog,
        run_function: Callable[[Dog], None],
) -> None:
    run_function(a_dog)
    
lassie: Dog = Dog()

make_dog_run(a_dog=lassie, run_function=animal_run)

An animal is running!


`animal_run` accepts an Animal, so it needs to work with a <b><font color='blue'>Dog</font></b>, since `Dog <: Animal`. Thus, it’s safe to use `animal_run` when `dog_run` is expected. Therefore, `Callable[[Animal], None] <: Callable[[Dog], None]`. As a consequence, <b>Callable is contravariant in the argument types</b>.

So now you understand what “function is covariant in the return type, but contravariant in the argument types” means. You nerd! 

In more general terms, anything that takes something— consumer, [sink](https://en.wikipedia.org/wiki/Sink_(computing)), reader, listener, etc. — is contravariant in the type of the objects it takes. It needs to be write-only with regard to the taken objects, though.

## <font color='darkblue'>Invariance</font>

### <font color='darkgreen'>Definition of Invariance</font>
An invariant type is neither covariant nor contravariant. More formally:
> Generic type `GenType[T, ...]` is invariant in type variable `T` if `GenType` is neither covariant in type variable `T` nor contravariant in type variable `T`.

### <font color='darkgreen'>List</font>
Yes, there’s no mistake, `List` is not like `Tuple`, which is covariant. <b>`List` is neither covariant nor contravariant. I will use this code to show it</b>:

In [14]:
class Animal:
    def eat(self):
        print('Om nom nom!')

class Dog(Animal):
    def bark(self):
        print('Woof woof!')

class Kangaroo(Animal):
    def leap(self):
        print('Wooooo!')

        
animals: List[Animal]
dogs: List[Dog]

All <b><font color='blue'>Animals</font></b>, including <b><font color='blue'>Dog</font></b>s and <b><font color='blue'>Kangaroo</font></b>s, can eat. Only <b><font color='blue'>Dog</font></b>s can bark, and only <b><font color='blue'>Kangaroo</font></b>s can leap.

Let’s get the obvious out of the way — `List` type is not contravariant. It’s not because you cannot safely assign animals to dogs, as animals may contain non-Dogs. It’s exactly the same as in `Tuple`’s case. Mypy agrees:
```python
dogs = animals  # mypy error: 
  # Incompatible types in assignment (expression has type 
  # "List[Animal]", variable has type "List[Dog]")
```
<br/>

Now, what about the covariance? If List was a covariant type, we could safely use a variable of the type `List[Dog]` instead of a variable of the type `List[Animal]`, yet we cannot:
```python
animals = dogs  # mypy error: 
  # Incompatible types in assignment (expression has type 
  # "List[Dog]", variable has type "List[Animal]")
```
<br/>

Why? Shouldn’t it be the same as in `Tuple`’s case as well?

Let’s step back a little and think what the difference between covariant `Tuple` type and `List` type is. Both are Sized, both are Iterables and both are Containers. Yet, <b>there is an essential difference between lists and tuples — mutability</b>. Python tuples are immutable — you cannot add or remove their elements. `List`s are mutable — you can add and remove their elements. It turns out that `List`’s mutability is something that determines `List` type not being covariant. If we used `List[Dog]` wherever `List[Animal]` is expected, we could append an incompatible object:

In [15]:
def append_kangaroo(animals: List[Animal]) -> None:
    animal = Kangaroo()
    # it's okay, since `Kangaroo <: Animal`
    animals.append(animal)

dogs: List[Dog] = []

lassie = Dog()
dogs.append(lassie)

append_kangaroo(dogs)

dogs[0].eat()  # fine...
# Om nom nom!

dogs[0].bark()  # fine...
# Woof woof!

dogs[1].eat()  # also fine...
# Om nom nom!

dogs[1].bark()  # oops!
# AttributeError: 'Kangaroo' object has no attribute 'bark'

Om nom nom!
Woof woof!
Om nom nom!


AttributeError: 'Kangaroo' object has no attribute 'bark'

So dogs list of the type `List[Dog]` got an object that cannot bark — a <b><font color='blue'>Kangaroo</font></b>. That’s why you cannot safely use object of `List[Dog]` type whenever there is an object of the type `List[Animal]` expected. Therefore `List[Dog]` is not a subtype of `List[Animal]`. Remember: the type is defined not only by a set of objects but also by functions/methods that can be used with these objects.

This is the same exploit that we discovered when discussing `Callable`. But `List`, contrary to `Callable`, is not contravariant either, as we've seen. It is invariant then:
> We neither have that `List[SubType] <: List[SuperType]` nor that `List[SuperType] <: List[SubType]`, for `SubType <: SuperType`.

Other invariant types are `Set`, `Dict`, and many more mutable containers. I think it is pretty obvious now and no further examples are needed. This feature of immutable containers is clearly advantageous over mutable ones. But that’s another story for another time…

In more general terms, any structure that supports both read and write operations on some set of objects — mutable container, queue, stack, heap, router, etc. — is invariant in the type of the objects it operates on.

## <font color='darkblue'>Bonus: Liskov Substitution Principle</font>
In this blog post series, I frequently said something like **it’s type-safe to use an object of the `SubType` type where an object of the `SuperType` type is expected**. This assumption is based on Liskov Substitution Principle — the principle formulated by Barbara Liskov, an American computer scientist. Wikipedia states it as follows:
> `[…]` if `S` is a subtype of `T`, then objects of type `T` may be replaced with objects of type `S` (<font color='brown'>i.e. an object of type `T` may be substituted with any object of a subtype `S`</font>) without altering any of the desirable properties of the program (correctness, task performed, etc.). [source](https://en.wikipedia.org/wiki/Liskov_substitution_principle)

This is a pretty strong requirement — when I replace any object of `SuperType` with an object of `SubType` (<font color='brown'>where `SubType <: SuperType`</font>) my program still needs to function correctly. It means not only that it cannot raise any new exceptions, but also it still needs to work according to its (<font color='brown'>original</font>) specification. This even means that we cannot override methods in subclasses if it changes the original behaviour; we can only add new methods. Python type system is not so rigid and [**mypy**](https://mypy.readthedocs.io/en/stable/) won’t complain when we override a method and change its behaviour (<font color='brown'>both methods— overridden and overriding — still need to have the same argument types, though!</font>). Yet, it’s good to know how to write even safer code